In [1]:
import codecs,unidecode,yaml,glob,os,shutil,re
import itertools as it
import numpy as np
import pandas as pd
# print(pd.__version__)

In [2]:
nQuizz=6
fMajeure=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Majeure.csv"
fMineure=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Mineure.csv"
fCM=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/CM.csv"
fAliasQDLE=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz-Alias.yaml"
fQuizzoodle=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz%02d.csv"%nQuizz
# fQuizzoodle=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/23-Quizz/Quizz%02d.csv"%nQuizz
fNotes=fQuizzoodle.replace(u".csv",u"-Notes.csv")

In [3]:
'''https://stackoverflow.com/a/35491326/3986209'''
def invert_dict(d): 
    inverse = dict() 
    for key in d: 
        # Go through the list that is saved in the dict:
        for item in d[key]:
            # Check if in the inverted dict the key exists
            if item not in inverse: 
                # If not create a new list
                inverse[item] = [key] 
            else: 
                inverse[item].append(key) 
    return inverse

# Lecture des données

In [4]:
try:
    with codecs.open(fAliasQDLE, 'r', encoding="utf8") as inFile:
        correspondances=yaml.safe_load(inFile)
    alias=correspondances["alias2etu"]
except:
    alias={}
alias

{'Aberard Florelle': ['440340'],
 'Adriana Mangiarratti': ['442806'],
 'Agathe raynal': ['436720'],
 'Angelina': ['438425'],
 'Angelina_Boukono': ['438425'],
 'Assimay nahi': ['436153'],
 'Audrey-luu': ['435643'],
 'Axelle-Meteau, Anais-Vieira': ['437803', '437028'],
 'BONNEAU MELISSA': ['439214'],
 'BONNEAU Melissa': ['439214'],
 u'BONNEAU-M\xe9lissa,TROUILLET-M': ['439214', '435953'],
 'Benhamouda-souha': ['442845'],
 'Boileau-Valentine': ['440209'],
 'Bordelait-Lizzie': ['439788'],
 'Boukang': ['438425'],
 'Boy-eleonore': ['441173'],
 'Camille Raynaud': ['438388'],
 'Camille raynaud': ['438388'],
 'Camille-Baldy': ['436384'],
 'Carla Rumeau': ['438098'],
 'Cassandra-clavereau': ['435774'],
 'Chanelle-Lacouture': ['441399'],
 'Collins-Amelia': ['437856'],
 u'Diallo M\u2019bamoussa': ['435468'],
 'Dombrovszky Yassine': ['442669'],
 u'D\u2019almeida': ['438740'],
 u'D\u2019almeida julienne': ['438740'],
 'Elya': ['440486'],
 'Emma Klopstein': ['428316'],
 'Erwan-Lecul': ['426946'],
 'E

In [5]:
len(alias)

217

In [6]:
colCM=[u"id",u"nom",u"prénom",u"email",u"ine"]
colQuizz=["pseudo","points","temps"]

try:
    dfCM=pd.read_csv(filepath_or_buffer=fCM,sep="\t",encoding="utf8",header=0,names=colCM)
    lQuizz=pd.read_csv(filepath_or_buffer=fQuizzoodle,sep=";",encoding="utf8",header=0,names=colQuizz)
    lQuizz.pseudo=lQuizz.pseudo.str.strip()
except:
    dfCM=None
    lQuizz=None

In [7]:
# lQuizz, dfCM

# Listage des alias

In [8]:
dfCM["PN"]=dfCM[u"prénom"].str.lower().str.strip()+" "+dfCM[u"nom"].str.lower().str.strip()
dfCM["NP"]=dfCM[u"nom"].str.lower().str.strip()+" "+dfCM[u"prénom"].str.lower().str.strip()
dfCM["P-N"]=dfCM[u"prénom"].str.lower().str.strip()+"-"+dfCM[u"nom"].str.lower().str.strip()
dfCM["N-P"]=dfCM[u"nom"].str.lower().str.strip()+"-"+dfCM[u"prénom"].str.lower().str.strip()
dfCM

,id,nom,prénom,email,ine,PN,NP,P-N,N-P
0,436893,ABELA--FRANZETTI,GABRIEL,gabriel.abela--franzetti@etu.u-bordeaux-montai...,1697705395,gabriel abela--franzetti,abela--franzetti gabriel,gabriel-abela--franzetti,abela--franzetti-gabriel
1,440340,ABERARD,FLORELLE,florelle.aberard@etu.u-bordeaux-montaigne.fr,1697705395,florelle aberard,aberard florelle,florelle-aberard,aberard-florelle
2,435916,ADAM,EULALIE,eulalie.adam@etu.u-bordeaux-montaigne.fr,1697705395,eulalie adam,adam eulalie,eulalie-adam,adam-eulalie
3,442931,AIT-SIDI-ALI,YASSINE,yassine.ait-sidi-ali@etu.u-bordeaux-montaigne.fr,1697705395,yassine ait-sidi-ali,ait-sidi-ali yassine,yassine-ait-sidi-ali,ait-sidi-ali-yassine
4,438199,AKPINAR,NESLIHAN,neslihan.akpinar@etu.u-bordeaux-montaigne.fr,1697705395,neslihan akpinar,akpinar neslihan,neslihan-akpinar,akpinar-neslihan
5,386382,ALVAREZ,ORIANE,oriane.alvarez@etu.u-bordeaux-montaigne.fr,1697705395,oriane alvarez,alvarez oriane,oriane-alvarez,alvarez-oriane
6,440881,AMOUZOU,RACHEL,rachel.amouzou@etu.u-bordeaux-montaigne.fr,1697705395,rachel amouzou,amouzou rachel,rachel-amouzou,amouzou-rachel
7,441950,ANDERSON,FREDERICK JOHN,frederick-john.anderson@etu.u-bordeaux-montaig...,1697705395,frederick john anderson,anderson frederick john,frederick john-anderson,anderson-frederick john
8,437791,APPRIOU,MAURINE,maurine.appriou@etu.u-bordeaux-montaigne.fr,1697705395,maurine appriou,appriou maurine,maurine-appriou,appriou-maurine
9,441237,ATAM-KASSIGADOU,ELIZA,eliza.atam-kassigadou@etu.u-bordeaux-montaigne.fr,1697705395,eliza atam-kassigadou,atam-kassigadou eliza,eliza-atam-kassigadou,atam-kassigadou-eliza


In [9]:
pseudos=lQuizz[~(lQuizz["pseudo"].isin(alias))]["pseudo"].str.strip().tolist()
print "\n".join(pseudos)

In [10]:
for nP,pseudoBrut in enumerate(pseudos):
    pseudo=unidecode.unidecode(pseudoBrut)
    print pseudo
    sPseudo = raw_input("Nom a chercher:\n")
    if sPseudo:
        print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]
        idPseudo=raw_input("id :\n")
        if idPseudo:
            alias[pseudoBrut]=idPseudo

In [11]:
# pseudo=u"csaintavi001 ebismes001"
# alias[pseudo]=["415962","413429"]

In [12]:
sPseudo="bisme"
print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]

Empty DataFrame
Columns: [PN, id]
Index: []


In [13]:
pseudos

[]

In [14]:
# alias["Sofiane"]

In [15]:
pn=dfCM["PN"].tolist()
p_n=dfCM["P-N"].tolist()
np=dfCM["NP"].tolist()
n_p=dfCM["N-P"].tolist()

In [16]:
for pseudo in pseudos:
    if unidecode.unidecode(pseudo.lower()) in pn:
        print pseudo,dfCM[dfCM["PN"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["PN"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]
    elif unidecode.unidecode(pseudo.lower()) in p_n:
        print pseudo,dfCM[dfCM["P-N"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["P-N"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]
    elif unidecode.unidecode(pseudo.lower()) in np:
        print pseudo,dfCM[dfCM["NP"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["NP"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]
    elif unidecode.unidecode(pseudo.lower()) in n_p:
        print pseudo,dfCM[dfCM["N-P"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["N-P"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]



In [17]:
for al in alias:
    if not isinstance(alias[al],list):
        if " " in alias[al]:
            alias[al]=alias[al].split(" ")
        else:
            alias[al]=[alias[al]]

In [18]:
len(alias)

217

In [19]:
invAlias=invert_dict(alias)
#invAlias[385732]=[u'adiarra001, cbeauvais004']
correspondances={"alias2etu":alias,"etu2alias":invAlias}
correspondances

{'alias2etu': {'Aberard Florelle': ['440340'],
  'Adriana Mangiarratti': ['442806'],
  'Agathe raynal': ['436720'],
  'Angelina': ['438425'],
  'Angelina_Boukono': ['438425'],
  'Assimay nahi': ['436153'],
  'Audrey-luu': ['435643'],
  'Axelle-Meteau, Anais-Vieira': ['437803', '437028'],
  'BONNEAU MELISSA': ['439214'],
  'BONNEAU Melissa': ['439214'],
  u'BONNEAU-M\xe9lissa,TROUILLET-M': ['439214', '435953'],
  'Benhamouda-souha': ['442845'],
  'Boileau-Valentine': ['440209'],
  'Bordelait-Lizzie': ['439788'],
  'Boukang': ['438425'],
  'Boy-eleonore': ['441173'],
  'Camille Raynaud': ['438388'],
  'Camille raynaud': ['438388'],
  'Camille-Baldy': ['436384'],
  'Carla Rumeau': ['438098'],
  'Cassandra-clavereau': ['435774'],
  'Chanelle-Lacouture': ['441399'],
  'Collins-Amelia': ['437856'],
  u'Diallo M\u2019bamoussa': ['435468'],
  'Dombrovszky Yassine': ['442669'],
  u'D\u2019almeida': ['438740'],
  u'D\u2019almeida julienne': ['438740'],
  'Elya': ['440486'],
  'Emma Klopstein': [

In [20]:
with codecs.open(fAliasQDLE, 'w', encoding="utf8") as outfile:
    yaml.safe_dump(correspondances, outfile, default_flow_style=False)

# Calcul des notes
Attention au nombre de questions

In [21]:
nbQ=7.0

In [22]:
def aliasQuizz(pseudo):
    result=None
    if pseudo in alias:
        result=alias[pseudo]
    return result

def noteQuizz(points):   
    result=8
    if points>0:
        result=int(round(10*(1+points/nbQ),0))
    return result
        
        

lQuizz["id"]=lQuizz["pseudo"].apply(aliasQuizz)
lQuizz["note"]=lQuizz["points"].apply(noteQuizz)

In [23]:
def splitPseudo1(id):
    result=None
    if id and len(id)>0:
        result=id[0]
    return result

def splitPseudo2(id):
    result=None
    if id and len(id)>1:
        result=id[1]
    return result

def splitPseudo3(id):
    result=None
    if id and len(id)>2:
        result=id[2]
    return result

lQuizz["id1"]=lQuizz["id"].apply(splitPseudo1)
lQuizz["id2"]=lQuizz["id"].apply(splitPseudo2)
lQuizz["id3"]=lQuizz["id"].apply(splitPseudo3)

lQuizz

,pseudo,points,temps,id,note,id1,id2,id3
0,rachel_amouzou,5,108.92,[440881],17,440881,None,None
1,Montoya-a Salome-a,4,109.65,"[438210, 438330]",16,438210,438330,None
2,NeuDea - LauPou,4,121.71,"[437937, 442121]",16,437937,442121,None
3,Eulalie-adam,3,75.96,[435916],14,435916,None,None
4,Maritxu.delas,3,88.50,[435616],14,435616,None,None
5,Diallo M’bamoussa,3,114.67,[435468],14,435468,None,None
6,Zoe-Roelandt,3,130.71,[438659],14,438659,None,None
7,olaia-yriarte,3,144.03,[437752],14,437752,None,None
8,Julia-Orlando,2,42.84,[438958],13,438958,None,None
9,raffy margot justine lacoume,2,42.87,"[441372, 437894]",13,441372,437894,None


In [24]:
df1=lQuizz[["pseudo","note","temps","id1"]].dropna()
df2=lQuizz[["pseudo","note","temps","id2"]].dropna()
df3=lQuizz[["pseudo","note","temps","id3"]].dropna()
df1.columns=["pseudo","note","temps","id"]
df2.columns=["pseudo","note","temps","id"]
df3.columns=["pseudo","note","temps","id"]

In [25]:
df123=pd.concat([df1,df2,df3]).sort_values(["note","temps"],ascending=[False,True])
df123

,pseudo,note,temps,id
0,rachel_amouzou,17,108.92,440881
1,Montoya-a Salome-a,16,109.65,438210
1,Montoya-a Salome-a,16,109.65,438330
2,NeuDea - LauPou,16,121.71,437937
2,NeuDea - LauPou,16,121.71,442121
3,Eulalie-adam,14,75.96,435916
4,Maritxu.delas,14,88.50,435616
5,Diallo M’bamoussa,14,114.67,435468
6,Zoe-Roelandt,14,130.71,438659
7,olaia-yriarte,14,144.03,437752


In [26]:
df123.loc[(df123.note==8)&(df123.temps>0),"note"]=10

In [32]:
df123.at[0, 'note'] = 20
if df123.loc[1, "note"].any()<18:
    df123.at[1, 'note'] = 18
df123

,pseudo,note,temps,id
0,rachel_amouzou,20,108.92,440881
1,Montoya-a Salome-a,18,109.65,438210
1,Montoya-a Salome-a,18,109.65,438330
2,NeuDea - LauPou,16,121.71,437937
2,NeuDea - LauPou,16,121.71,442121
3,Eulalie-adam,14,75.96,435916
4,Maritxu.delas,14,88.50,435616
5,Diallo M’bamoussa,14,114.67,435468
6,Zoe-Roelandt,14,130.71,438659
7,olaia-yriarte,14,144.03,437752


In [33]:
dfNotes=df123.groupby("id").max().sort_values("note",ascending=False)
dfNotes.to_csv(fNotes,sep="\t",encoding="utf8")

In [34]:
dfNotes

,pseudo,note,temps
id,,,
440881,rachel_amouzou,20,108.92
438330,Montoya-a Salome-a,18,109.65
438210,Montoya-a Salome-a,18,109.65
442121,NeuDea - LauPou,16,121.71
437937,NeuDea - LauPou,16,121.71
437752,olaia-yriarte,14,144.03
435468,Diallo M’bamoussa,14,114.67
438659,Zoe-Roelandt,14,130.71
435616,Maritxu.delas,14,88.50
